### FOR BODY DAMAGE

### IMAGE PRE PROCESSING

### 1. Import The ImageDataGenerator Library

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### 2. Configure ImageDataGenerator Class

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./225,shear_range = 0.1, zoom_range = 0.1,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)


### 3. Apply ImageDataGenerator Functionality To Trainset And Testset

In [4]:
training_set = train_datagen.flow_from_directory(r"/content/drive/MyDrive/Projects/Car damage/body/training",target_size = (224,224),  class_mode = "categorical",batch_size = 10)
test_set = test_datagen.flow_from_directory(r"/content/drive/MyDrive/Projects/Car damage/body/validation",target_size = (224,224),  class_mode = "categorical",batch_size = 10)

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


### MODEL BUILDING

### 1. Importing The Model Building Libraries

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

### 2. Loading The Model

In [6]:
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/Projects/Car damage/body/training'
valid_path = '/content/drive/MyDrive/Projects/Car damage/body/validation'

In [7]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


### 3. Adding Flatten Layer

In [8]:
for layer in vgg16.layers:
    layer.trainable = False

In [9]:
folders = glob('/content/drive/MyDrive/Projects/Car damage/body/training/*')

In [10]:
folders

['/content/drive/MyDrive/Projects/Car damage/body/training/01-rear',
 '/content/drive/MyDrive/Projects/Car damage/body/training/02-side',
 '/content/drive/MyDrive/Projects/Car damage/body/training/00-front']

In [11]:
x = Flatten()(vgg16.output)

In [12]:
len(folders)

3

### 4. Adding Output Layer

In [13]:
prediction = Dense(len(folders), activation='softmax')(x)

### 5. Creating A Model Object

In [14]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### 6. Configure The Learning Process

In [15]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

### 7. Train The Model 

In [ ]:
r = model.fit_generator(training_set,validation_data=test_set,epochs=25,steps_per_epoch=len(training_set),validation_steps=len(test_set))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/25
98/98 [==============================] - 545s 6s/step - loss: 1.2961 - accuracy: 0.5271 - val_loss: 1.3222 - val_accuracy: 0.5263
Epoch 2/25
98/98 [==============================] - 523s 5s/step - loss: 0.6537 - accuracy: 0.7354 - val_loss: 0.8519 - val_accuracy: 0.6491
Epoch 3/25
98/98 [==============================] - 522s 5s/step - loss: 0.5249 - accuracy: 0.7937 - val_loss: 0.9558 - val_accuracy: 0.6374
Epoch 4/25
98/98 [==============================] - 522s 5s/step - loss: 0.3639 - accuracy: 0.8693 - val_loss: 1.0587 - val_accuracy: 0.6316
Epoch 5/25
98/98 [==============================] - 521s 5s/step - loss: 0.3006 - accuracy: 0.8979 - val_loss: 0.9209 - val_accuracy: 0.6901
Epoch 6/25
98/98 [==============================] - 523s 5s/step - loss: 0.2114 - accuracy: 0.9377 - val_loss: 0.9228 - val_accuracy: 0.6725
Epoch 7/25
98/98 [==============================] - 523s 5s/step - loss: 0.1560 - accuracy: 0.9530 - val_loss: 0.9612 - val_accuracy: 0.6725
Epoch 8/25
98

### 8. Save The Model

In [ ]:
cd /content/drive/MyDrive/Projects

/content/drive/MyDrive/Projects


In [ ]:
from tensorflow.keras.models import load_model

model.save('body.h5')

### 9. Test The Model

In [ ]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [ ]:
cd /content/drive/MyDrive/Projects

/content/drive/MyDrive/Projects


In [ ]:
model = load_model('body.h5')

In [ ]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds

In [ ]:
import numpy as np

In [ ]:
data = "/content/drive/MyDrive/Projects/Car damage/body/training/00-front/0010.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 924ms/step
front


### FOR LEVEL DAMAGE

### 1. Import The ImageDataGenerator Library

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### 2. Configure ImageDataGenerator Class

In [17]:
train_datagen = ImageDataGenerator(rescale = 1./225,shear_range = 0.1, zoom_range = 0.1,horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)


### 3. Apply ImageDataGenerator Functionality To Trainset And Testset

In [18]:
training_set = train_datagen.flow_from_directory(r"/content/drive/MyDrive/Projects/Car damage/level/training",target_size = (224,224),  class_mode = "categorical",batch_size = 10)
test_set = test_datagen.flow_from_directory(r"/content/drive/MyDrive/Projects/Car damage/level/validation",target_size = (224,224),  class_mode = "categorical",batch_size = 10)

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


### MODEL BUILDING

### 1. Importing The Model Building Libraries

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

### 2. Loading The Model

In [20]:
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/Projects/Car damage/level/training'
valid_path = '/content/drive/MyDrive/Projects/Car damage/level/validation'

In [21]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

### 3. Adding Flatten Layer

In [22]:
for layer in vgg16.layers:
    layer.trainable = False

In [23]:
folders = glob('/content/drive/MyDrive/Projects/Car damage/body/training/*')

In [24]:
folders

['/content/drive/MyDrive/Projects/Car damage/body/training/01-rear',
 '/content/drive/MyDrive/Projects/Car damage/body/training/02-side',
 '/content/drive/MyDrive/Projects/Car damage/body/training/00-front']

In [25]:
x = Flatten()(vgg16.output)

In [26]:
len(folders)

3

### 4. Adding Output Layer

In [27]:
prediction = Dense(len(folders), activation='softmax')(x)

### 5. Creating A Model Object

In [28]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [29]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

### 6. Configure The Learning Process

In [30]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

### 7. Train The Model

In [31]:
r = model.fit(training_set,validation_data=test_set,epochs=10,steps_per_epoch=len(training_set),validation_steps=len(test_set))

Epoch 1/10
98/98 [==============================] - 622s 6s/step - loss: 1.1873 - accuracy: 0.5557 - val_loss: 1.1638 - val_accuracy: 0.6082
Epoch 2/10
98/98 [==============================] - 606s 6s/step - loss: 0.7436 - accuracy: 0.7222 - val_loss: 1.0043 - val_accuracy: 0.6433
Epoch 3/10
98/98 [==============================] - 607s 6s/step - loss: 0.4893 - accuracy: 0.8131 - val_loss: 1.5654 - val_accuracy: 0.5789
Epoch 4/10
98/98 [==============================] - 605s 6s/step - loss: 0.3905 - accuracy: 0.8560 - val_loss: 1.0406 - val_accuracy: 0.5965
Epoch 5/10
98/98 [==============================] - 611s 6s/step - loss: 0.2691 - accuracy: 0.9081 - val_loss: 1.0311 - val_accuracy: 0.6550
Epoch 6/10
98/98 [==============================] - 608s 6s/step - loss: 0.2242 - accuracy: 0.9193 - val_loss: 1.3213 - val_accuracy: 0.6140
Epoch 7/10
98/98 [==============================] - 617s 6s/step - loss: 0.3779 - accuracy: 0.8662 - val_loss: 2.4248 - val_accuracy: 0.5263
Epoch 8/10
98

### 8. Save The Model

In [32]:
cd /content/drive/MyDrive/Projects

/content/drive/MyDrive/Projects


In [33]:
from tensorflow.keras.models import load_model

model.save('level.h5')

### 9. Test The Model

In [34]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [35]:
model = load_model('level.h5')

In [36]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds

In [37]:
import numpy as np

In [38]:
data = "/content/drive/MyDrive/Projects/Car damage/level/validation/03-severe/0067.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 747ms/step
rear
